# 7 Ordenación

El problema de ordenar un conjunto de datos (por ejemplo, en orden ascendente) tiene gran importancia tanto teórica como práctica. En esta sección veremos principalmente algoritmos que ordenan mediante comparaciones entre llaves, para los cuales se puede demostrar una cota inferior que coincide con la cota superior provista por varios algoritmos. También veremos un algoritmo de otro tipo, que al no hacer comparaciones, no está sujeto a esa cota inferior.

## Cota inferior

Supongamos que deseamos ordenar tres datos $A$, $B$ y $C$. La siguiente figura muestra un árbol de decisión posible para resolver este problema. Los nodos internos del árbol representan comparaciones y los nodos externos representan salidas emitidas por el programa.

![arbol-decision-ordenacion](arbol-decision-ordenacion.gif)

Como se vio en el capítulo de búsqueda, todo árbol de decisión con $H$ hojas tiene al menos altura $\log_2{H}$, y la altura del árbol de decisión es igual al número de comparaciones que se efectúan en el peor caso.

En un árbol de decisión para ordenar $n$ datos se tiene que $H=n!$, y por lo tanto se tiene que todo algoritmo que ordene $n$ datos mediante comparaciones entre llaves debe hacer al menos $\log_2{n!}$ comparaciones en el peor caso.

Usando la aproximación de Stirling, se puede demostrar que $\log_2{n!} = n \log_2{n} + \Theta(n)$, por lo cual la cota inferior es de $\Theta(n\log{n})$.

Si suponemos que todas las posibles permutaciones resultantes son equiprobables, es posible demostrar también que el número promedio de comparaciones que cualquier algoritmo debe hacer es también de $\Theta(n\log{n})$.

## Quicksort

Este método fue inventado por C.A.R. Hoare a comienzos de los '60s, y sigue siendo el método más eficiente para uso general.

Quicksort es un ejemplo clásico de la aplicación del principio de *dividir para reinar*. Su estructura es la siguiente:

* Primero se elige un elemento al azar, que se denomina el pivote.

* El arreglo a ordenar se reordena dejando a la izquierda a los elementos menores que el pivote, el pivote al medio, y a la derecha los elementos mayores que el pivote:

![particion](particion.gif)

* Luego cada sub-arreglo se ordena recursivamente.

La recursividad termina, en principio, cuando se llega a sub-arreglos de tamaño cero o uno, los cuales trivialmente ya están ordenados. En la práctica veremos que es preferible detener la recursividad antes de eso, para no desperdiciar tiempo ordenando recursivamente arreglos pequeños, los cuales pueden ordenarse más eficientemente usando Ordenación por Inserción, por ejemplo.

In [15]:
def quicksort(a):
    qs(a,0,len(a)-1)

def qs(a,i,j): # ordena a[i],...,a[j]
    if i<j: # quedan 2 o más elementos por ordenar
        k=particion(a,i,j)
        qs(a,i,k-1)
        qs(a,k+1,j)

def particion(a,i,j): # particiona a[i],...,a[j], retorna en k posición del pivote
    k=np.random.randint(i,j) # genera un número al azar k en i..j
    (a[i],a[k])=(a[k],a[i]) # mueve a[k] al extremo izquierdo
    # a[i] es el pivote
    s=i # invariante: a[i+1..s]<=p, a[s+1..t]>p
    for t in range(s,j):
        if a[t+1]<=a[i]:
            (a[s+1],a[t+1])=(a[t+1],a[s+1])
            s=s+1
    # mover pivote al centro
    (a[i],a[s])=(a[s],a[i])
    return s

In [21]:
import numpy as np
a = np.random.random(6)
print(a)
quicksort(a)
print(a)

[0.20423073 0.20768937 0.81650528 0.1804613  0.71118114 0.54139117]
[0.1804613  0.20423073 0.20768937 0.54139117 0.71118114 0.81650528]


### Costo promedio de Quicksort

Si suponemos, como una primera aproximación, que el pivote siempre resulta ser la mediana del conjunto, entonces el costo de ordenar está dado (aproximadamente) por la ecuación de recurrencia

$$
T(n)=n+2T\left( \frac{n}{2} \right)
$$

Esto tiene solución $T(n) = n \log_2{n}$ y es, en realidad, el *mejor* caso de Quicksort.

Para analizar el tiempo promedio que demora la ordenación mediante Quicksort, observemos que el funcionamiento de Quicksort puede graficarse mediante un *árbol de partición*:

![arbol-particion](arbol-particion.gif)

Por la forma en que se construye, es fácil ver que el árbol de partición es un *árbol de búsqueda binaria*, y como el pivote es escogido al azar, entonces la raíz de cada subárbol puede ser cualquiera de los elementos del conjunto en forma equiprobable. En consecuencia, los árboles de partición y los árboles de búsqueda binaria tienen exactamente la misma distribución.

En el proceso de partición, cada elemento de los subárboles ha sido comparado contra la raíz (el pivote). Al terminar el proceso, cada elemento ha sido comparado contra todos sus ancestros. Si sumamos todas estas comparaciones, el resultado total es igual al *largo de caminos internos*.

Usando todas estas correspondencias, tenemos que, usando los resultados ya conocidos para árboles, el número promedio de comparaciones que realiza Quicksort es de:

$$
T(n)=1.38 n\log_2{n}+\Theta(n)
$$

Por lo tanto, Quicksort, en el caso esperado, corre en un tiempo proporcional a la cota inferior.

### Peor caso de Quicksort

El peor caso de Quicksort se produce cuando el pivote resulta ser siempre el mínimo o el máximo del conjunto. En este caso la ecuación de recurrencia es

$$
T(n) = n - 1 + T(n-1)
$$
            
lo que tiene solución $T(n) = \Theta(n^2)$. Desde el punto de vista del árbol de partición, esto corresponde a un árbol en "zig-zag".

Si bien este peor caso es extremadamente improbable si el pivote se escoge al azar, algunas implementaciones de Quicksort toman como pivote al primer elemento del arreglo (suponiendo que, al venir el arreglo al azar, entonces el primer elemento es tan aleatorio como cualquier otro). El problema es que si el conjunto viene en realidad ordenado, entonces caemos justo en el peor caso cuadrático.

Lo anterior refuerza la importancia de que el pivote se escoja al azar. Esto no aumenta significativamente el costo total, porque el número total de elecciones de pivote es $\Theta(n)$.

### Mejoras a Quicksort

Quicksort puede ser optimizado de varias maneras, pero hay que ser muy cuidadoso con estas mejoras, porque es fácil que terminen empeorando el desempeño del algoritmo.

En primer lugar, es desaconsejable hacer cosas que aumenten la cantidad de trabajo que se hace dentro del "loop" de partición, porque este es el lugar en donde se concentra el costo $\Theta(n \log{n})$.

Algunas de las mejoras que han dado buen resultado son las siguientes:

#### Quicksort con "mediana de 3"

En esta variante, el pivote no se escoge como un elemento tomado al azar, sino que primero se extrae una muestra de 3 elementos, y entre ellos se escoge a la mediana de esa muestra como pivote.

Si la muestra se escoge tomando al primer elemento del arreglo, al del medio y al último, entonces lo que era el peor caso (arreglo ordenado) se transforma de inmediato en mejor caso.

De todas formas, es aconsejable que la muestra se escoja al azar, y en ese caso el análisis muestra que el costo esperado para ordenar n elementos es

$$
\frac{12}{7} n \ln{n}  \approx  1.19 n \log_2{n}
$$

Esta reducción en el costo se debe a que el pivote es ahora una mejor aproximación a la mediana. De hecho, si en lugar de escoger una muestra de tamaño 3, lo hiciéramos con tamaños como 7, 9, etc., se lograría una reducción aún mayor, acercándonos cada vez más al óptimo, pero con rendimientos rápidamente decrecientes.

#### Uso de Ordenación por Inserción para ordenar sub-arreglos pequeños

Tal como se dijo antes, no es eficiente ordenar recursivamente sub-arreglos demasiado pequeños.

En lugar de esto, se puede establecer un tamaño mínimo $M$, de modo que los sub-arreglos de tamaño menor que esto se ordenan por inserción en lugar de por Quicksort.

Claramente debe haber un valor óptimo para $M$, porque si creciera indefinidamente se llegaría a un algoritmo cuadrático. Esto se puede analizar, y el óptimo es cercano a 10.

Como método de implementación, al detectarse un sub-arreglo de tamaño menor que $M$, se lo puede dejar simplemente sin ordenar, retornando de inmediato de la recursividad. Al final del proceso, se tiene un arreglo cuyos pivotes están en orden creciente, y encierran entre ellos a bloques de elementos desordenados, pero que ya están en el grupo correcto. Para completar la ordenación, entonces, basta con hacer una sola gran pasada de Ordenación por Inserción, la cual ahora no tiene costo $\Theta(n^2)$, sino $\Theta(nM)$, porque ningún elemento está a distancia mayor que $M$ de su ubicación definitiva.

#### Ordenar recursivamente sólo el sub-arreglo más pequeño

Un problema potencial con Quicksort es la profundidad que puede llegar a tener la recursividad. En el peor caso, ésta puede llegar a ser $\Theta(n)$.

Para evitar esto, se puede usar recursividad solo para la "mitad" más pequeña, y ordenar la otra "mitad" de manera iterativa. Con este enfoque, cada llamada recursiva se aplica a un sub-arreglo cuyo tamaño es a lo más la mitad del tamaño del arreglo a ordenar, de modo que si llamamos $S(n)$ a la profundidad de recursión, tenemos que

$$
S(n) \le 1 + S\left(\frac{n}{2}\right)
$$

lo cual tiene solución $\log_2{n}$, de modo que la profundidad de la recursión nunca es más que logarítmica.

In [29]:
def qs(a,i,j): # ordena a[i],...,a[j]
    while i<j: # quedan 2 o más elementos por ordenar
        k=particion(a,i,j)
        if k-i<=j-k: #mitad izquierda es más pequeña
            qs(a,i,k-1)
            i=k+1
        else:
            qs(a,k+1,j)
            j=k-1

In [30]:
import numpy as np
a = np.random.random(6)
print(a)
quicksort(a)
print(a)

[0.56382863 0.00164504 0.70130608 0.15055317 0.97007422 0.93080778]
[0.00164504 0.15055317 0.56382863 0.70130608 0.93080778 0.97007422]
